In [ ]:
pip install llama_index

In [ ]:
!pip install colab-env


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!wget "https://moitt.gov.pk/SiteImage/Misc/files/Final%20Draft%20Personal%20Data%20Protection%20Bill%20May%202023.pdf" -O pak.pdf
!wget "https://www.paisayaar.pk/h5/privacy.html" -O paisayaar.html

--2024-12-17 23:58:07--  https://moitt.gov.pk/SiteImage/Misc/files/Final%20Draft%20Personal%20Data%20Protection%20Bill%20May%202023.pdf
Resolving moitt.gov.pk (moitt.gov.pk)... 203.101.184.86, 2407:9e00:50:1:1:1:a1:5
Connecting to moitt.gov.pk (moitt.gov.pk)|203.101.184.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660065 (645K) [application/pdf]
Saving to: ‘pak.pdf’

pak.pdf             100%[===================>] 644.59K   592KB/s    in 1.1s    

2024-12-17 23:58:09 (592 KB/s) - ‘pak.pdf’ saved [660065/660065]

--2024-12-17 23:58:09--  https://www.paisayaar.pk/h5/privacy.html
Resolving www.paisayaar.pk (www.paisayaar.pk)... 175.107.194.125
Connecting to www.paisayaar.pk (www.paisayaar.pk)|175.107.194.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36973 (36K) [text/html]
Saving to: ‘paisayaar.html’

paisayaar.html      100%[===================>]  36.11K  --.-KB/s    in 0s      

2024-12-17 23:58:10 (244 MB/s) - ‘paisayaar.ht

In [ ]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["pak.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
print(nodes[5].get_content(metadata_mode="all"))

page_label: 6
file_name: pak.pdf
file_path: pak.pdf
file_type: application/pdf
file_size: 660065
creation_date: 2024-12-17
last_modified_date: 2023-05-19

5 
 
PERSONAL DATA PROTECTION BILL, 2023 
 
The Personal Data Protection Bill, 2023 is devised to regulate the collection, processing, use, 
disclosure, and transfer of personal data and additionally provides a data protection mechanism 
including the offences concerning the violation of data privacy rights of an individual.    
 
Where a person collects, processes, stores, uses, and discloses data, it must respect the rights, 
freedoms, and dignity of an individual for matters connected therewith and ancillary thereto. 
Therefore, the Bill is enacted as follows:


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [ ]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [ ]:
llm = OpenAI(model="gpt-4o", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool],
    "What are the major points of pak.pdf as described on page 20?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "major points", "page_numbers": ["20"]}
=== Function Output ===
The major points in the provided context include the conditions under which consent for processing personal data is not required, such as for legal proceedings, medical purposes, protecting vital interests, obtaining legal advice, establishing legal rights, administration of justice, and exercising functions conferred by law. Additionally, it mentions the possibility of excluding certain clauses through a Gazette order and imposing further conditions as specified in the order.


In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of pak.pdf"
    ),
)



In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents_2 = SimpleDirectoryReader(input_files=["paisayaar_privacy_policy.pdf"]).load_data()
splitter_2 = SentenceSplitter(chunk_size=1024)
nodes_2 = splitter_2.get_nodes_from_documents(documents_2)

vector_index = VectorStoreIndex(nodes_2)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

summary_index_2 = SummaryIndex(nodes_2)
summary_query_engine_2 = summary_index_2.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_paisayaar = QueryEngineTool.from_defaults(
    name="summary_tool_paisayaar",
    query_engine=summary_query_engine_2,
    description=(
        "Useful if you want to get a summary of paisayaar_privacy_policy.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_paisayaar],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool, summary_tool_paisayaar],
    "Which pdf is longer pak or paisayaar?",
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "pak"}
=== Function Output ===
The 'paisayaar_privacy_policy.pdf' document is partially compliant with the Pakistan Data Protection Law (PDPL). It addresses several key areas but has notable gaps that need to be addressed to achieve full compliance. The non-compliance points include issues with data retention and deletion policies, data security measures, international data transfers, data breach notification procedures, user rights, and consent management. Actionable recommendations have been provided to address these non-compliance points and improve alignment with the PDPL.


In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)
print(agent_worker)
print(agent)

In [ ]:
response = agent.query(
    "Tell me about the 3rd party data collection as mentioned in pak.pdf"
)

Added user message to memory: Tell me about the 3rd party data collection as mentioned in pak.pdf
=== Calling Function ===
Calling function: vector_tool with args: {"query": "3rd party data collection", "page_numbers": []}
=== Function Output ===
The regulations outlined in the provided context do not specifically address third-party data collection.
=== LLM Response ===
The provided context from pak.pdf does not specifically address third-party data collection. If you have any other questions or need further information, feel free to ask!


In [ ]:
response = agent.chat(
  "Are there any policies for 3rd part data in the doc pak.pdf"
)

Added user message to memory: Are there any policies for 3rd part data in the doc pak.pdf
=== Calling Function ===
Calling function: vector_tool with args: {"query": "third-party data policies", "page_numbers": []}
=== Function Output ===
The Bill addresses the regulation of personal data collection, processing, use, disclosure, and transfer, emphasizing the importance of respecting individuals' rights, freedoms, and dignity. It outlines the procedures for handling personal data by government, organizations, and individuals, ensuring fair practices in the digital economy. Additionally, the Bill aims to provide extra protection for children's data and emphasizes that personal data should only be collected through lawful, fair, and consensual means and used for the intended purposes or directly related purposes.
=== LLM Response ===
The document outlines regulations for the collection, processing, use, disclosure, and transfer of personal data, emphasizing lawful, fair, and consensual me

In [ ]:
response = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Analyze and extract compliance points which an application's privacy policy must address as required by the Pakistan Data Protection Law (PDPL). "
    "Focus on the following key compliance areas strictly based on the PDPL: "
    "1. Data Collection and Notification: Verify if the privacy policy specifies the types of personal data collected and whether the data subject is informed about the purpose, legal basis, and methods of collection. "
    "2. Purpose Limitation: Ensure the policy defines specific purposes for data processing and does not allow processing beyond these purposes. "
    "3. Consent and Legal Basis: Evaluate whether the policy meets PDPL’s consent requirements, including explicit consent for sensitive data, and allows for easy consent withdrawal. "
    "4. Data Subject Rights: Check if the policy outlines the rights of data subjects, including access, rectification, erasure, restriction of processing, and the right to object to automated decisions. "
    "5. Data Retention: Analyze whether the policy explains the retention period for personal data and compliance with PDPL’s requirement to not retain data longer than necessary. "
    "6. Data Sharing and Transfers: Identify if the policy mentions sharing or transferring data to third parties domestically or internationally and includes safeguards for international transfers. "
    "7. Security Measures: Review the technical and organizational measures outlined to ensure the security and confidentiality of personal data. "
    "8. Data Breach Notification: Verify whether the policy mentions the notification process for data breaches, including informing authorities and data subjects. "
    "9. Grievance Mechanism: Confirm if a clear mechanism is provided for users to lodge complaints or contact a Data Protection Officer (DPO), if applicable. "
    "10. Data Localization: Check compliance with any requirements for storing and processing Pakistani users' data within local servers as required by the PDPL. "
    "Your response should clearly highlight each compliance point's presence, absence, or adequacy in the privacy policy and recommend necessary improvements for full compliance with the PDPL."
)


Added user message to memory: You are an expert in data protection laws and compliance. Analyze and extract compliance points which an application's privacy policy must address as required by the Pakistan Data Protection Law (PDPL). Focus on the following key compliance areas strictly based on the PDPL: 1. Data Collection and Notification: Verify if the privacy policy specifies the types of personal data collected and whether the data subject is informed about the purpose, legal basis, and methods of collection. 2. Purpose Limitation: Ensure the policy defines specific purposes for data processing and does not allow processing beyond these purposes. 3. Consent and Legal Basis: Evaluate whether the policy meets PDPL’s consent requirements, including explicit consent for sensitive data, and allows for easy consent withdrawal. 4. Data Subject Rights: Check if the policy outlines the rights of data subjects, including access, rectification, erasure, restriction of processing, and the right

In [ ]:
response = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. "
    "Review the privacy policy comprehensively and identify its alignment with these compliance requirements. "
    "Your response should include: "
    "1. A summary of key compliance obligations derived from the PDPL. "
    "2. A detailed analysis of whether the privacy policy meets these obligations. "
    "3. Any areas of non-compliance or gaps observed, with recommendations for improvement. "
    "Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis."
)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. Review the privacy policy comprehensively and identify its alignment with these compliance requirements. Your response should include: 1. A summary of key compliance obligations derived from the PDPL. 2. A detailed analysis of whether the privacy policy meets these obligations. 3. Any areas of non-compliance or gaps observed, with recommendations for improvement. Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "Pakistan Data Protection Law (PDPL) compliance obligations for privacy policy"}
=== Function Output ===
The Pakistan Data Protection Law (PDPL) outlines several compliance obligations for privacy policy, includin

In [ ]:
# Install required libraries and wkhtmltopdf
!apt-get install -y wkhtmltopdf
!pip install pdfkit

import pdfkit

# Configure path to wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

# URL of the privacy policy
url = "https://www.paisayaar.pk/h5/privacy.html"

# Output PDF file name
output_file = "paisayaar_privacy_policy.pdf"

# Convert the webpage to PDF
pdfkit.from_url(url, output_file, configuration=config)

print(f"Privacy policy saved as {output_file}")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Privacy policy saved as paisayaar_privacy_policy.pdf


In [ ]:
response = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. "
    "Review the privacy policy saved as paisayaar_privacy_policy.pdf comprehensively and identify its alignment with these compliance requirements. "
    "Your response should include: "
    "1. A summary of key compliance obligations derived from the PDPL. "
    "2. A detailed analysis of whether the privacy policy meets these obligations. "
    "3. Any areas of non-compliance or gaps observed, with recommendations for improvement. "
    "Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis."
)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address. Review the privacy policy saved as paisayaar_privacy_policy.pdf comprehensively and identify its alignment with these compliance requirements. Your response should include: 1. A summary of key compliance obligations derived from the PDPL. 2. A detailed analysis of whether the privacy policy meets these obligations. 3. Any areas of non-compliance or gaps observed, with recommendations for improvement. Focus only on PDPL's legal and regulatory requirements while providing a clear and structured analysis.
=== LLM Response ===
To perform a comprehensive analysis of the privacy policy saved as "paisayaar_privacy_policy.pdf" against the Pakistan Data Protection Law (PDPL), I would need access to the content of that specific document. Since I can't directly

In [ ]:
Points = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address."
    "Structure it in a bullet point list"

)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address.Structure it in a bullet point list
=== LLM Response ===
Here are the mandatory compliance points that an application's privacy policy must address according to the Pakistan Data Protection Law (PDPL), structured in a bullet point list:

- **Consent**:
  - Obtain explicit consent from data subjects for processing personal data.
  - Ensure specific consent for processing sensitive data.
  - Provide mechanisms for easy withdrawal of consent.

- **Notice and Transparency**:
  - Clearly inform data subjects about the types of personal data collected.
  - Specify the purposes for data collection and processing.
  - Disclose the legal basis for processing personal data.
  - Explain the methods of data collection.

- **Data Subject Rights**:
  - Right to acc

In [ ]:
Points = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address."
    "Structure it in a bullet point lis"
    "Your job is to then format and make it into a to do list against which every data protection law will be tested"

)


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address.Structure it in a bullet point lisYour job is to then format and make it into a to do list against which every data protection law will be tested
=== LLM Response ===
Here's a to-do list formatted to test an application's privacy policy against the mandatory compliance points derived from the Pakistan Data Protection Law (PDPL):

### PDPL Compliance To-Do List for Privacy Policy

- [ ] **Consent Management**
  - [ ] Obtain explicit consent from data subjects for processing personal data.
  - [ ] Ensure specific consent for processing sensitive data.
  - [ ] Provide mechanisms for easy withdrawal of consent.

- [ ] **Notice and Transparency**
  - [ ] Clearly inform data subjects about the types of personal data collected.
  - [ ] Specify the purposes f

In [ ]:
# Points = agent.chat(
#     "You are an expert in data protection laws and compliance. "
#     "Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address."
#     "Structure it in a bullet point list"
#     "Your job is to then format and make it into a to do list against which every data protection law will be tested"
#     "Now that you have the to do list your job is to pick the paisayaar_privacy_policy.pdf and see how well does it comply to these points. Generate details about what the privacy policy document say about each point"
#     "Return this report and also your decision on whether the privacy policy is compliant to the data protection law or not. Along with this also give me points on which the report is not compliant"
# )


Added user message to memory: You are an expert in data protection laws and compliance. Carefully analyze the Pakistan Data Protection Law (PDPL) and derive all mandatory compliance points that an application's privacy policy must address.Structure it in a bullet point lisYour job is to then format and make it into a to do list against which every data protection law will be testedNow that you have the to do list your job is to pick the paisayaar_privacy_policy.pdf and see how well does it comply to these points. Generate details about what the privacy policy document say about each pointReturn this report and also your decision on whether the privacy policy is compliant to the data protection law or not. Along with this also give me points on which the report is not compliant
=== Calling Function ===
Calling function: vector_tool with args: {"query": "paisayaar_privacy_policy.pdf compliance with PDPL", "page_numbers": []}
=== Function Output ===
The paisayaar_privacy_policy.pdf needs 

In [ ]:
print(type(Points.response))

<class 'str'>


In [ ]:
from fpdf import FPDF
from google.colab import files


# Initialize PDF object
pdf = FPDF()
pdf.add_page()

txt = Points.response

# Set font for the PDF
pdf.set_font("Arial", size=12)

# Write the content line by line
for line in txt.split("\n"):
    pdf.cell(200, 10, txt=line, ln=True, align="L")

# Save the PDF to a file
output_file = "report.pdf"
pdf.output(output_file)

# Download the PDF file
files.download(output_file)

print(f"PDF saved successfully and downloaded as '{output_file}'")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF saved successfully and downloaded as 'report.pdf'


In [ ]:
from fpdf import FPDF
from google.colab import files

# Function to generate a PDF for a given text
def generate_pdf(text, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(190, 10, txt=text)
    pdf.output(filename)
    files.download(filename)
    print(f"PDF generated and downloaded: {filename}")
# generate_pdf(Stage1.response, "Stage1_PDPL_Compliance_Points.pdf")

# Stage 1: Derive Compliance Points from PDPL
Stage1 = agent.chat(
    "You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). "
    "Your task is to thoroughly review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. "
    "Provide a detailed and comprehensive bullet-point list of these compliance requirements, ensuring each point is clear and actionable."
)
generate_pdf(Stage1.response, "Stage1_PDPL_Compliance_Points.pdf")

# Stage 2: Create a PDPL Compliance To-Do List
Stage2 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the compliance points derived in Stage 1, transform them into a to-do list format. "
    "This checklist should be actionable and structured in a way that each point can be used to evaluate a privacy policy's alignment with the PDPL."
)
generate_pdf(Stage2.response, "Stage2_PDPL_ToDo_List.pdf")

# Stage 3: Analyze the Privacy Policy Document
Stage3 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the to-do list created in Stage 2, analyze the 'paisayaar_privacy_policy.pdf' document. "
    "For each compliance point, provide a detailed analysis of what the privacy policy states and assess whether it aligns with the PDPL. "
    "Highlight specific clauses or sections in the privacy policy corresponding to each compliance point."
)
generate_pdf(Stage3.response, "Stage3_PDPL_Policy_Analysis.pdf")

# Stage 4: Generate the Compliance Report
Stage4 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Stage 3. "
    "Clearly state whether the privacy policy is fully compliant, partially compliant, or non-compliant with the PDPL. "
    "Provide a list of non-compliance points, along with detailed explanations of how the policy fails to meet these requirements. "
    "Include actionable recommendations to address any gaps or shortcomings in the privacy policy to ensure full compliance with the PDPL."
)
generate_pdf(Stage4.response, "Stage4_PDPL_Compliance_Report.pdf")


Added user message to memory: You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to thoroughly review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. Provide a detailed and comprehensive bullet-point list of these compliance requirements, ensuring each point is clear and actionable.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "pak.pdf"}
=== Function Output ===
The information provided does not contain any details or references related to a document named "pak.pdf."
=== Calling Function ===
Calling function: vector_tool with args: {"query": "mandatory compliance requirements for privacy policy", "page_numbers": []}
=== Function Output ===
Mandatory compliance requirements for a privacy policy include addressing international data transfers by incorporating user consent and protection measures, establishing clear proce

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage1_PDPL_Compliance_Points.pdf
Added user message to memory: You are an expert in data protection laws and compliance. Using the compliance points derived in Stage 1, transform them into a to-do list format. This checklist should be actionable and structured in a way that each point can be used to evaluate a privacy policy's alignment with the PDPL.
=== LLM Response ===
Here is an actionable to-do list to evaluate a privacy policy's alignment with the Pakistan Data Protection Law (PDPL):

1. **International Data Transfers**:
   - [ ] Verify that the privacy policy includes a section on international data transfers.
   - [ ] Check if user consent is explicitly obtained for international data transfers.
   - [ ] Ensure that protection measures for international data transfers are clearly outlined.

2. **Data Breach Notifications**:
   - [ ] Confirm that the privacy policy outlines procedures for data breach notifications.
   - [ ] Ensure that the policy s

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage3_PDPL_Policy_Analysis.pdf
Added user message to memory: You are an expert in data protection laws and compliance. Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Stage 3. Clearly state whether the privacy policy is fully compliant, partially compliant, or non-compliant with the PDPL. Provide a list of non-compliance points, along with detailed explanations of how the policy fails to meet these requirements. Include actionable recommendations to address any gaps or shortcomings in the privacy policy to ensure full compliance with the PDPL.
=== LLM Response ===
**Compliance Status of 'paisayaar_privacy_policy.pdf': Partially Compliant**

**Non-Compliance Points:**

1. **International Data Transfers**:
   - **Explanation**: The privacy policy does not mention international data transfers or the requirement for user consent related to such transfers. This is a critical aspect of the PDPL, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Stage4_PDPL_Compliance_Report.pdf


In [ ]:
Stage1 = agent.chat(
    "You are an expert in data protection laws and compliance, with a specialization in the Pakistan Data Protection Law (PDPL). "
    "Carefully review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. "
    "Provide a clear, detailed, and comprehensive bullet-point list of these compliance requirements. "
    "Ensure that each compliance point is actionable and unambiguous, covering key areas such as: "
    "- Data collection practices "
    "- User consent mechanisms "
    "- User rights (e.g., access, correction, erasure) "
    "- Data sharing with third parties "
    "- Data retention and deletion policies "
    "- Data security measures "
    "- International data transfers "
    "Include specific references to the clause, page, and line number in the PDPL for each compliance requirement."
)

generate_pdf(Stage1.response, "2_Stage1_PDPL_Compliance_Points.pdf")


Added user message to memory: You are an expert in data protection laws and compliance, with a specialization in the Pakistan Data Protection Law (PDPL). Carefully review the PDPL and extract all mandatory compliance requirements that an application's privacy policy must address. Provide a clear, detailed, and comprehensive bullet-point list of these compliance requirements. Ensure that each compliance point is actionable and unambiguous, covering key areas such as: - Data collection practices - User consent mechanisms - User rights (e.g., access, correction, erasure) - Data sharing with third parties - Data retention and deletion policies - Data security measures - International data transfers Include specific references to the clause, page, and line number in the PDPL for each compliance requirement.
=== Calling Function ===
Calling function: summary_tool with args: {"input": "pak.pdf"}
=== Function Output ===
The Personal Data Protection Bill, 2023 in Pakistan aims to regulate the c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage1_PDPL_Compliance_Points.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage1_PDPL_Compliance_Points.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_1 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_1",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage1_PDPL_Compliance_Points.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_Stage_1],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage2 = agent.chat(
    "Using the compliance points derived in Stage 1 (Saved in the pdf '2_Stage1_PDPL_Compliance_Points.pdf'), create a structured and actionable compliance to-do list. "
    "This checklist should be organized into categories that align with the PDPL requirements (e.g., data collection, user consent, security, retention). "
    "Each task in the to-do list should be concise, testable, and include a reference to the relevant PDPL clause, page, and line number. "
    "This to-do list will serve as a framework for evaluating privacy policies for compliance with the PDPL."
)

generate_pdf(Stage2.response, "2_Stage2_PDPL_ToDo_List.pdf")


Added user message to memory: Using the compliance points derived in Stage 1 (Saved in the pdf '2_Stage1_PDPL_Compliance_Points.pdf'), create a structured and actionable compliance to-do list. This checklist should be organized into categories that align with the PDPL requirements (e.g., data collection, user consent, security, retention). Each task in the to-do list should be concise, testable, and include a reference to the relevant PDPL clause, page, and line number. This to-do list will serve as a framework for evaluating privacy policies for compliance with the PDPL.
=== Calling Function ===
Calling function: summary_tool_Stage_1 with args: {"input": "Create a structured and actionable compliance to-do list based on the compliance points derived in Stage 1, organized into categories that align with the PDPL requirements."}
=== Function Output ===
Create a structured and actionable compliance to-do list based on the compliance points derived in Stage 1, organized into categories th

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage2_PDPL_ToDo_List.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage2_PDPL_ToDo_List.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_2 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_2",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage2_PDPL_ToDo_List.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool, summary_tool_Stage_1, summary_tool_Stage_2],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage3 = agent.chat(
    "You are an expert in data protection laws and compliance. "
    "Using the PDPL compliance to-do list created in Stage 2 (saved as 2_Stage2_PDPL_ToDo_List.pdf), analyze the 'paisayaar_privacy_policy.pdf' document. "
    "For each compliance point, provide: "
    "- A detailed explanation of what the privacy policy states about that specific point. "
    "- An assessment of whether the privacy policy meets the PDPL requirement. "
    "- Specific references to the clause, page, and line number in the PDPL that the compliance point corresponds to. "
    "- Highlight any areas of non-compliance and explain the gaps in detail. "
    "Ensure that all references to PDPL clauses are accurate and specific."
)

generate_pdf(Stage3.response, "2_Stage3_PDPL_Policy_Analysis.pdf")


Added user message to memory: You are an expert in data protection laws and compliance. Using the PDPL compliance to-do list created in Stage 2 (saved as 2_Stage2_PDPL_ToDo_List.pdf), analyze the 'paisayaar_privacy_policy.pdf' document. For each compliance point, provide: - A detailed explanation of what the privacy policy states about that specific point. - An assessment of whether the privacy policy meets the PDPL requirement. - Specific references to the clause, page, and line number in the PDPL that the compliance point corresponds to. - Highlight any areas of non-compliance and explain the gaps in detail. Ensure that all references to PDPL clauses are accurate and specific.
=== Calling Function ===
Calling function: summary_tool_Stage_2 with args: {"input": "Please summarize the PDPL compliance to-do list from Stage 2."}
=== Function Output ===
The PDPL compliance to-do list from Stage 2 includes tasks related to Data Retention and Deletion Policies, Data Security Measures, Intern

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage3_PDPL_Policy_Analysis.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage3_PDPL_Policy_Analysis.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_3 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_3",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage3_PDPL_Policy_Analysis.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
Stage4 = agent.chat(
    "Generate a comprehensive compliance report based on the analysis performed in Stage 3 (saved as 2_Stage3_PDPL_Policy_Analysis.pdf) of the 'paisayaar_privacy_policy.pdf' document. "
    "The report must include: "
    "1. A clear summary of the compliance status, indicating whether the privacy policy is fully compliant, partially compliant, or non-compliant with the Pakistan Data Protection Law (PDPL). "
    "2. A detailed list of non-compliance points, including: "
    "   - A description of how the policy fails to meet the PDPL requirement. "
    "   - References to the specific PDPL clause, page, and line number being violated. "
    "3. Actionable recommendations for addressing each area of non-compliance to achieve full alignment with the PDPL. "
    "Structure the report clearly with headings and bullet points for easy readability."
)



generate_pdf(Stage4.response, "2_Stage4_PDPL_Compliance_Report.pdf")


Added user message to memory: Generate a comprehensive compliance report based on the analysis performed in Stage 3 (saved as 2_Stage3_PDPL_Policy_Analysis.pdf) of the 'paisayaar_privacy_policy.pdf' document. The report must include: 1. A clear summary of the compliance status, indicating whether the privacy policy is fully compliant, partially compliant, or non-compliant with the Pakistan Data Protection Law (PDPL). 2. A detailed list of non-compliance points, including:    - A description of how the policy fails to meet the PDPL requirement.    - References to the specific PDPL clause, page, and line number being violated. 3. Actionable recommendations for addressing each area of non-compliance to achieve full alignment with the PDPL. Structure the report clearly with headings and bullet points for easy readability.
=== Calling Function ===
Calling function: summary_tool_Stage_3 with args: {"input": "Generate a comprehensive compliance report based on the analysis performed in Stage 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage4_PDPL_Compliance_Report.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.node_parser import SentenceSplitter

# load documents
documents = SimpleDirectoryReader(input_files=["2_Stage4_PDPL_Compliance_Report.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)



def vector_query(
    query: str,
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool_Stage_4 = QueryEngineTool.from_defaults(
    name="summary_tool_Stage_4",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of 2_Stage4_PDPL_Compliance_Report.pdf"
    ),
)

llm = OpenAI(model="gpt-4o", temperature=0)

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool,summary_tool_Stage_1, summary_tool_Stage_2, summary_tool_Stage_3, summary_tool_Stage_4],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

ImportError: cannot import name 'SimpleDirectoryReader' from 'llama_index' (unknown location)

In [ ]:
Stage5 = agent.chat(
    "You are an expert in compliance validation with a specialization in data protection laws, particularly the Pakistan Data Protection Law (PDPL). "
    "Your task is to thoroughly validate the compliance report generated in Stage 4 (saved as '2_Stage4_PDPL_Compliance_Report.pdf') to ensure its accuracy, consistency, and reliability. "
    "Follow the steps below and provide a structured, clear, and actionable validation output: "

    "### Step 1: Cross-Check PDPL References ### "
    "For each compliance point and non-compliance point mentioned in the report: "
    "1. Verify whether the referenced PDPL clause, page, and line number actually exist in the PDPL document. "
    "2. Quote the exact wording of the relevant clause from the PDPL. "
    "3. Compare this clause with the corresponding section or wording in the privacy policy document. Quote the exact wording from the privacy policy for clarity. "
    "4. Provide a structured comparison: "
    "   - Compliance Point: [Describe the compliance point from the report] "
    "   - PDPL Requirement (Quoted): [Quote the exact PDPL clause] "
    "   - Privacy Policy Content (Quoted): [Quote what the privacy policy says] "
    "   - Assessment: [State whether the compliance/non-compliance determination is accurate based on this comparison] "

    "### Step 2: Validate the Assessment ### "
    "Carefully review whether the report's assessment of compliance or non-compliance is accurate. "
    "Confirm that the reasoning aligns with both the PDPL requirements and the privacy policy's content. Highlight any discrepancies found."

    "### Step 3: Evaluate Recommendations ### "
    "Assess the recommendations provided in the compliance report: "
    "1. Verify whether the recommendations are actionable, clear, and directly address the non-compliance or gaps identified. "
    "2. Suggest improvements to any recommendations that are unclear or insufficient for achieving full compliance. "

    "### Step 4: Validation Summary ### "
    "Summarize the overall validation results, providing: "
    "1. A clear statement on whether the compliance report is accurate, consistent, and complete. "
    "2. A list of any errors, inaccuracies, or missing references found in the report. "
    "3. Suggested corrections or improvements to ensure the report meets the highest standard of accuracy and reliability. "

    "### Format Your Response ### "
    "Structure your response into the following sections: "
    "1. Validation of PDPL References "
    "2. Validation of Compliance Assessments "
    "3. Evaluation of Recommendations "
    "4. Final Validation Summary "

    "Be thorough, precise, and clear in your analysis. Use structured bullet points and headings for readability, ensuring that the final output can serve as a reliable validation of the compliance report."
)


generate_pdf(Stage4.response, "2_Stage5_PDPL_Compliance_Report.pdf")


Added user message to memory: You are an expert in compliance validation with a specialization in data protection laws, particularly the Pakistan Data Protection Law (PDPL). Your task is to thoroughly validate the compliance report generated in Stage 4 (saved as '2_Stage4_PDPL_Compliance_Report.pdf') to ensure its accuracy, consistency, and reliability. Follow the steps below and provide a structured, clear, and actionable validation output: ### Step 1: Cross-Check PDPL References ### For each compliance point and non-compliance point mentioned in the report: 1. Verify whether the referenced PDPL clause, page, and line number actually exist in the PDPL document. 2. Quote the exact wording of the relevant clause from the PDPL. 3. Compare this clause with the corresponding section or wording in the privacy policy document. Quote the exact wording from the privacy policy for clarity. 4. Provide a structured comparison:    - Compliance Point: [Describe the compliance point from the report]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: 2_Stage5_PDPL_Compliance_Report.pdf


In [ ]:
# response = agent.chat(
#     "You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to assess the compliance of the 'paisayaar_privacy_policy.pdf' document with the PDPL. Follow these steps systematically and produce a structured output for each: "



#     "### Step 1: Derive Compliance Points from PDPL ### "
#     "Carefully analyze the PDPL to extract all mandatory compliance requirements that an application's privacy policy must address. "
#     "Provide a detailed and actionable bullet-point list covering key areas such as: "
#     "- Data collection practices "
#     "- User consent mechanisms "
#     "- User rights (e.g., access, correction, erasure) "
#     "- Data sharing with third parties "
#     "- Data retention and deletion policies "
#     "- Data security measures "
#     "- International data transfers "
#     "Include specific references to the clause, page, and line number from the PDPL for each compliance point. "

#     "### Step 2: Create a PDPL Compliance To-Do List ### "
#     "Based on the compliance points derived in Step 1, create a structured and actionable compliance to-do list. "
#     "Organize it into categories (e.g., data collection, user consent, security, retention) with concise, testable tasks. "
#     "Ensure each item includes a reference to the relevant clause, page, and line number in the PDPL. "

#     "### Step 3: Analyze the Privacy Policy Document ### "
#     "Using the compliance to-do list from Step 2, evaluate the 'paisayaar_privacy_policy.pdf' document. For each compliance point: "
#     "- Provide a detailed explanation of what the privacy policy states about that point. "
#     "- Assess whether the privacy policy meets the PDPL requirement. "
#     "- Reference specific clauses or sections in the privacy policy relevant to each compliance point. "
#     "- Highlight non-compliance areas with detailed explanations and include references to the relevant clause, page, and line number in the PDPL. "

#     "### Step 4: Generate the Compliance Report ### "
#     "Summarize the compliance status of the 'paisayaar_privacy_policy.pdf' document based on the analysis performed in Step 3. "
#     "The report must include: "
#     "1. A clear summary of the compliance status: fully compliant, partially compliant, or non-compliant. "
#     "2. A detailed list of non-compliance points, explaining how the policy fails to meet PDPL requirements, with references to the specific clause, page, and line number. "
#     "3. Actionable recommendations for addressing each area of non-compliance to achieve full PDPL compliance. "
#     "Structure the report clearly with headings and bullet points for easy readability. "

#     "### Step 5: Validate the Compliance Report ### "
#     "Carefully validate the compliance report generated in Step 4. "
#     "For each compliance and non-compliance point, cross-check the findings with the PDPL document to ensure accuracy and consistency. "
#     "Confirm that the referenced clauses, pages, and lines are correct, and verify the alignment of the assessment with the privacy policy content. "
#     "Provide a validation summary stating whether the report is accurate and complete. If any corrections or improvements are needed, include those as well."
# )

# generate_pdf(Stage4.response, "Run_Poc.pdf")



Added user message to memory: You are an expert in data protection laws and compliance, specializing in the Pakistan Data Protection Law (PDPL). Your task is to assess the compliance of the 'paisayaar_privacy_policy.pdf' document with the PDPL. Follow these steps systematically and produce a structured output for each: ### Step 1: Derive Compliance Points from PDPL ### Carefully analyze the PDPL to extract all mandatory compliance requirements that an application's privacy policy must address. Provide a detailed and actionable bullet-point list covering key areas such as: - Data collection practices - User consent mechanisms - User rights (e.g., access, correction, erasure) - Data sharing with third parties - Data retention and deletion policies - Data security measures - International data transfers Include specific references to the clause, page, and line number from the PDPL for each compliance point. ### Step 2: Create a PDPL Compliance To-Do List ### Based on the compliance points

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF generated and downloaded: Run_Poc.pdf
